In [1]:
import sympy
import numpy as np
import isotropic_tensors as it

its = it.IsotropicTensorSystem(usecache=True)

Please have a look at [tensor algebra notebook](./tensor_algebra.ipynb) to get some understanding of the isotropic tensor before reading this

The distribution of e.g. the tidal tensor is given by
$$p(\mathbf{T}) = N \exp \left(- \frac{1}{2} \mathbf{T}^T \mathbf{C}_{\mathbf{T}}^+ \mathbf{T} \right)$$
where it is shown in the [tensor algebra notebook](./tensor_algebra.ipynb) how to understand $\mathbf{C}_{\mathbf{T}}^+$. <br> Bias parameters are given by:
$$b_{\mathbf{J}_{X}} = (-1)^N \left\langle\frac{1}{p} \frac{\partial p}{\partial \mathbf{T}^N} \cdot^{(2N)} \frac{\mathbf{J}_{X}}{||{ \mathbf{J}_X }||^2}\right\rangle_g$$
The first and second derivative of $p$ are:
$$\frac{1}{p} \frac{\partial p}{\partial \mathbf{T}} =  - \mathbf{C}_{\mathbf{T}}^+ \mathbf{T}$$
$$\frac{1}{p} \frac{\partial^2 p}{\partial \mathbf{T}^2} = (\mathbf{T} \mathbf{C}_{\mathbf{T}}^+) \otimes (\mathbf{C}_{\mathbf{T}}^+ \mathbf{T}) - \mathbf{C}_{\mathbf{T}}^+$$
Therefore, we just need to contract these derivatives with the isotropic tensor of interest to get a bias estimator

In [2]:
print("If we consider only the distribution of the tidal tensor (as written above), then the derivative is")
display(its.symbol_bias_deriv1("J2", potderiv=(2,)))

If we consider only the distribution of the tidal tensor (as written above), then the derivative is


J_2*T/sigma0**2

However, for the joint distribution of second ($T$) and fourth ($R$) derivatives, we have
$$b_{J_2} = -\left(\begin{matrix}\frac{15 \sigma_{2}^{2} J_{2=2}}{2 \sigma_{0}^{2} \sigma_{2}^{2} - 2 \sigma_{1}^{4}} + \frac{\sigma_{2}^{2} J_{22}}{\sigma_{0}^{2} \sigma_{2}^{2} - \sigma_{1}^{4}} & \frac{15 \sigma_{1}^{2} J_{2=4}}{2 \sigma_{0}^{2} \sigma_{2}^{2} - 2 \sigma_{1}^{4}} + \frac{\sigma_{1}^{2} J_{24}}{\sigma_{0}^{2} \sigma_{2}^{2} - \sigma_{1}^{4}}\\\frac{15 \sigma_{1}^{2} J_{4=2}}{2 \sigma_{0}^{2} \sigma_{2}^{2} - 2 \sigma_{1}^{4}} + \frac{\sigma_{1}^{2} J_{42}}{\sigma_{0}^{2} \sigma_{2}^{2} - \sigma_{1}^{4}} & \frac{15 \sigma_{0}^{2} J_{4=4}}{2 \sigma_{0}^{2} \sigma_{2}^{2} - 2 \sigma_{1}^{4}} + \frac{\sigma_{0}^{2} J_{44}}{\sigma_{0}^{2} \sigma_{2}^{2} - \sigma_{1}^{4}} + \frac{315 J_{4==4}}{8 \sigma_{2}^{2}}\end{matrix}\right)
\begin{pmatrix} T \\ R \end{pmatrix}
\cdot^{(2)} \frac{J_2}{||J_2||}
$$ 
(Omitting the indicators of the expectation value)<br>
This evaluates to:

In [3]:
display(its.symbol_bias_deriv1("J2", potderiv=(2,4)))

(sigma1**2*J_4*R + sigma2**2*J_2*T)/(sigma0**2*sigma2**2 - sigma1**4)

Where $J_2 T = \delta$ and $J_4 R = \nabla^2 \delta = L$. So the estimator depends on whether higher spatial derivatives variables are considered. <br> We can get all other terms in a similar manner. I have written some functions which do this for us

In [4]:
deriv1 = its.symbol_bias_deriv1()
deriv2 = its.symbol_bias_deriv2()

allterm = {**deriv1, **deriv2}

In [5]:
mat = []
for term in allterm:
    if "3" in term:
        nospatial = allterm[term]
    else:
        nospatial = sympy.together(allterm[term].subs(its.sigma[1], 0)) # A hack to see the term if the covariance between 2nd and 4th derivatives were 0
    
    mat.append([its(term).symbol(), nospatial, allterm[term]])
mat = sympy.Matrix(mat)
mat

Matrix([
[       J_2,                                     J_2*T/sigma0**2,                                                                                                                                                       (sigma1**2*J_4*S + sigma2**2*J_2*T)/(sigma0**2*sigma2**2 - sigma1**4)],
[       J_4,                                     J_4*S/sigma2**2,                                                                                                                                                       (sigma0**2*J_4*S + sigma1**2*J_2*T)/(sigma0**2*sigma2**2 - sigma1**4)],
[      J_22,                  (-sigma0**2 + J_22*T**2)/sigma0**4,                                (sigma1**4*J_44*S**2 + sigma1**2*sigma2**2*J_24*T*S + sigma1**2*sigma2**2*J_42*S*T + sigma2**4*J_22*T**2 - sigma2**2*(sigma0**2*sigma2**2 - sigma1**4))/(sigma0**2*sigma2**2 - sigma1**4)**2],
[     J_2=2,      15*(-2*sigma0**2 + 3*J_2=2*T**2)/(4*sigma0**4),           15*(3*sigma1**4*J_4=4*S**2 + 3*sigma1**2*sigma2**2*

Here the first column indicates the isotropic tensor associated with the bias term of interest, the second column the lower spatial order estimator and the last column the spatial order (up to) 4 estimator. <br>
sympy inevitably miswrites some terms here, e.g. $J_{22} T^2$ should actually be $T J_{22} T$, simply corresponding to $\delta^2$. Since we anyways contract all tensors fully with the corresponding tensors, we might as well skip writing the tensor:

In [6]:
nicesubs = [(its.x[0], 1), (its.x[1], 1), (its.x[2], 1)]
nicesubs += [(its.sigstar4, sympy.Symbol("sigma_*", real=True, positive=True)**4)]
nicesubs += [(its("J42").symbol(), its("J24").symbol())] 
nicesubs += [(its("J4=2").symbol(), its("J2=4").symbol())]

In [7]:
mat = []
for term in allterm:
    if "3" in term:
        nospatial = allterm[term]
    else:
        nospatial = sympy.together(allterm[term].subs(its.sigma[1], 0))
    
    mat.append([its(term).symbol(), nospatial.subs(nicesubs), its.symbol_collect_Js(sympy.together(allterm[term].subs(nicesubs)))])
mat = sympy.Matrix(mat)
mat

Matrix([
[       J_2,                                  J_2/sigma0**2,                                                                                                           (J_2*sigma2**2 + J_4*sigma1**2)/sigma_***4],
[       J_4,                                  J_4/sigma2**2,                                                                                                           (J_2*sigma1**2 + J_4*sigma0**2)/sigma_***4],
[      J_22,                  (-sigma0**2 + J_22)/sigma0**4,                                                     (J_22*sigma2**4 + 2*J_24*sigma1**2*sigma2**2 + J_44*sigma1**4 - sigma2**2*sigma_***4)/sigma_***8],
[     J_2=2,      15*(-2*sigma0**2 + 3*J_2=2)/(4*sigma0**4),                                     15*(3*J_2=2*sigma2**4 + 6*J_2=4*sigma1**2*sigma2**2 + 3*J_4=4*sigma1**4 - 2*sigma2**2*sigma_***4)/(4*sigma_***8)],
[      J_24,                     J_24/(sigma0**2*sigma2**2),                     (J_22*sigma1**2*sigma2**2 + J_24*(sigma0**2*sigma2**2 + sigma1

Where each $J$ term is supposed to be replaced by a corresponding $J-$ contraction of the appropriate potential-derivative

# Python Code
Finally, we create python code for evaluating the computed estimators.

In [8]:
with open("bias_estimators.py", "w") as f:
    f.write("import numpy as np\n\n")
    f.write('bias_estimators = {2:{}, 4:{}}\n\n')
    f.write("#---------- Spatial Order 2 -----------\n\n")
    
    for term in "J2", "J22", "J2=2":
        pycode, fname = its.pycode_bias_estimator(term, potderiv=(2,), label="so2", getinfo=True)
        f.write(pycode + "\n")
        f.write('bias_estimators[2]["%s"] = %s\n\n' % (term, fname))
    
    f.write("#---------- Spatial Order 4 -----------\n")
    for term in "J2", "J22", "J2=2", "J4", "J24", "J2=4", "J3-3", "J3---3", "J44", "J4=4", "J4==4":
        pycode, fname = its.pycode_bias_estimator(term, potderiv=(2,3,4), label="so4", getinfo=True)
        f.write(pycode + "\n")
        f.write('bias_estimators[4]["%s"] = %s\n\n' % (term,fname))

Have a look at bias_estimators.py to see the generated code.
Note that we count here spatial orders in terms of derivatives from the potential (rather from the density). For the conventional labelling subtract two from the order that is stated here

# A Toy model as example
Let's say we had a density field with variance $\sigma_0=1$ and a set of tracers defined through $0 < \delta < 1$. What is their bias $b_1$?

In [9]:
import bias_estimators as be

In [10]:
sigma = 1.
delta = np.random.normal(0., sigma, size=(10000))
tracer_mask = (delta > 0.) & (delta < 1.)

# All estimator function assume two inputs
# (1): the variables as a dictionary, where each key is the name
#      of an isotropic tensor and the value is the tensor contracted
#      with the corresponding derivatives of the potential
# (2): A list of the spectral moments of the linear density field
#      sigmas = np.sqrt([np.mean(delta**2), -np.mean(delta*L), np.mean(L**2), ...])
#      Which terms are needed depends on the term and the order of the estimator
terms = {"J2": delta[tracer_mask]}
sigmas = [sigma]
b1_per_object = be.bias_estimators[2]["J2"](terms, sigmas)
b1 = np.mean(b1_per_object)
b2_per_object = be.bias_estimators[2]["J22"](terms, sigmas)
b2 = np.mean(b2_per_object)
print(b1, b2)

0.4574828065161686 -0.7110443922515665
